In [1]:
from pyspark.sql import types as T
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from datetime import datetime, timedelta
import random
import json

In [2]:
spark = SparkSession.builder.appName("DataFrameComJson").getOrCreate()

schema = T.StructType([
    T.StructField("id", T.IntegerType(), nullable=False),
    T.StructField("data_processamento", T.TimestampType(), nullable=False),
    T.StructField("nome_cliente", T.StringType(), nullable=False),
    T.StructField("sobrenome", T.StringType(), nullable=False),
])

data = [
    (
        i,
        datetime.now() - timedelta(days=random.randint(0, 30)),
        '  maria  ',
        '  Santos  '
    )
    for i in range(1, 10)
]

df = spark.createDataFrame(data, schema=schema)

In [3]:
df.show(5, False)

+---+--------------------------+------------+----------+
|id |data_processamento        |nome_cliente|sobrenome |
+---+--------------------------+------------+----------+
|1  |2025-04-25 12:02:21.903655|  maria     |  Santos  |
|2  |2025-05-12 12:02:21.90368 |  maria     |  Santos  |
|3  |2025-04-30 12:02:21.903683|  maria     |  Santos  |
|4  |2025-04-25 12:02:21.903685|  maria     |  Santos  |
|5  |2025-05-15 12:02:21.903688|  maria     |  Santos  |
+---+--------------------------+------------+----------+
only showing top 5 rows



In [4]:
def add_columns(df, columns_add):
    return df.withColumns(columns_add)

In [5]:
col_to_trim = {
    "nome_cliente_trim": F.trim(F.col("nome_cliente"))
}

col_to_upper = {
    "nome_cliente_upper": F.upper(F.col("nome_cliente"))
}

col_to_upper_trim = {
    "nome_cliente_upper_trim": F.trim(F.upper(F.col("nome_cliente")))
}

In [6]:
df_f = (
    df.transform(add_columns, col_to_trim)
    .transform(add_columns, col_to_upper)
    .transform(add_columns, col_to_upper_trim)
)

df_f = df_f.withColumn('cont_chars_nome_cliente_upper', F.length(F.col('nome_cliente_upper')))
df_f = df_f.withColumn('cont_chars_nome_cliente_trim', F.length(F.col('nome_cliente_trim')))
df_f = df_f.withColumn('cont_chars_nome_cliente_upper_trim', F.length(F.col('nome_cliente_upper_trim')))

In [7]:
(
    df_f
    .select(
        'id',
        'nome_cliente',
        'cont_chars_nome_cliente_upper',
        'nome_cliente_trim',
        'cont_chars_nome_cliente_trim',
        'nome_cliente_upper_trim',
        'cont_chars_nome_cliente_upper_trim'
    ).toPandas()
    .head()
)

,id,nome_cliente,cont_chars_nome_cliente_upper,nome_cliente_trim,cont_chars_nome_cliente_trim,nome_cliente_upper_trim,cont_chars_nome_cliente_upper_trim
0,1,maria,9,maria,5,MARIA,5
1,2,maria,9,maria,5,MARIA,5
2,3,maria,9,maria,5,MARIA,5
3,4,maria,9,maria,5,MARIA,5
4,5,maria,9,maria,5,MARIA,5


In [8]:
# withColumns
col_to_trim = {
    "nome_cliente_upper": F.upper(F.col("nome_cliente")),
    "sobrenome_upper": F.upper(F.col("sobrenome"))
}

dd = df.withColumns(col_to_trim)
dd.toPandas().head()

,id,data_processamento,nome_cliente,sobrenome,nome_cliente_upper,sobrenome_upper
0,1,2025-04-25 12:02:21.903655,maria,Santos,MARIA,SANTOS
1,2,2025-05-12 12:02:21.903680,maria,Santos,MARIA,SANTOS
2,3,2025-04-30 12:02:21.903683,maria,Santos,MARIA,SANTOS
3,4,2025-04-25 12:02:21.903685,maria,Santos,MARIA,SANTOS
4,5,2025-05-15 12:02:21.903688,maria,Santos,MARIA,SANTOS
